```
conda init
conda create -n get-face-embeddings python=3.10 -c conda -y
conda activate get-face-embeddings
pip install torch==2.2.1+cu118 torchvision==0.17.1+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
pip install facenet-pytorch==2.6.0
pip install ipykernel
```

In [1]:
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
import torchvision.transforms as T

c:\Users\ExoHorizon\miniconda3\envs\get-face-embeddings\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [3]:
# MTCNN | Face Detection & Alignment
mtcnn = MTCNN(
    image_size=160, 
    margin=0, 
    min_face_size=20, 
    device=device
    )

# Backup | Face Detection & Alignment
fallback_preprocess = T.Compose([
    T.Resize((160, 160)), 
    T.ToTensor()
    ])

# FaceNet | Face Embedding Model
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Freeze FaceNet Parameters
for param in facenet.parameters():
    param.requires_grad = False

In [4]:
def load_images(image_paths):
    """
    Loads a list of PIL images from file paths.

    Args:
        image_paths (list of str): List of file paths to the images.

    Returns:
        list of PIL.Image: List of loaded images.
    """
    images = [Image.open(p).convert("RGB") for p in image_paths]
    return images

In [5]:
def detect_and_align_faces(images):
    """
    Detects and aligns faces in a list of images using MTCNN.
    Args:
        images (list of PIL.Image): List of images to process.
    Returns:
        torch.Tensor: Batched (N, 3, 160, 160) tensor of aligned face images.
    """
    aligned_faces = []
    for img in images:
        with torch.no_grad():
            face = mtcnn(img)
        if face is None:
            print("MTCNN failed, using fallback preprocessing.")
            face = fallback_preprocess(img)
        aligned_faces.append(face)

    batched_faces = torch.stack(aligned_faces).to(device)
    batched_faces.requires_grad_(False)
    return batched_faces

In [6]:
def get_facenet_embeddings(batched_faces):
    """
    Computes FaceNet embeddings for a batch of aligned face images.

    Args:
        batched_faces (torch.Tensor): Batched (N, 3, 160, 160) tensor of aligned face images.

    Returns:
        torch.Tensor: (N, 512) tensor of FaceNet embeddings.
    """
    with torch.no_grad():
        embeddings = facenet(batched_faces)
        embeddings = torch.nn.functional.normalize(embeddings, dim=1)
    return embeddings

In [7]:
# Example Usage
paths = ['examples/arnold.jpg']
images = load_images(paths)
batched_faces = detect_and_align_faces(images)
embeddings = get_facenet_embeddings(batched_faces)
print("Batch embeddings shape:", embeddings.shape)
print("Batch requires grad:", batched_faces.requires_grad)
print(embeddings)

# Note: Will have to store face embeddings on disk.

Batch embeddings shape: torch.Size([1, 512])
Batch requires grad: False
tensor([[ 1.9267e-02, -6.2522e-02, -2.7900e-02, -1.3280e-02,  2.8392e-02,
         -2.2933e-02,  2.3828e-02,  3.0381e-03, -4.8157e-02,  3.0667e-03,
         -2.3772e-02, -4.4999e-03, -2.2181e-02, -1.5663e-02, -1.7127e-02,
         -1.9158e-02, -1.0076e-02, -3.4952e-03, -4.0085e-02, -1.8336e-03,
         -6.5617e-02,  7.2723e-02, -4.8383e-02, -4.1195e-02, -6.7140e-02,
         -2.7650e-02, -2.6945e-02, -3.7456e-02,  6.0905e-02,  2.3177e-02,
          3.6290e-02, -3.4735e-02,  7.1270e-02, -4.3447e-02,  2.2719e-02,
         -7.1355e-02, -9.8319e-03, -4.1455e-02,  2.7450e-02, -1.2223e-02,
         -6.7260e-02,  2.1217e-02,  5.2414e-02, -3.8535e-02,  1.7849e-03,
          8.9962e-03, -1.8462e-03, -9.4337e-03,  2.1585e-02,  3.6563e-02,
          3.2678e-02,  8.2742e-02,  1.6761e-02, -3.2673e-02,  6.9676e-03,
         -1.6004e-02,  5.2109e-03, -1.0909e-02, -2.4560e-02,  6.7624e-02,
          8.1146e-03, -4.7215e-03,  3.63